<a href="https://colab.research.google.com/github/CSID-DGU/2024-2-DSCD-Fiveguys-6/blob/main/Genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 도로 기반 경로 사용
!pip install osmnx

In [17]:
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import pandas as pd

# **출근**

# 데이터 불러오기

In [27]:
df_morning = pd.read_csv('/content/drive/MyDrive/출근_정류장_좌표_수요.csv', encoding='CP949')
df_morning = df_morning[['ARS_ID', '출근시간_수요', '정류소명', 'X좌표', 'Y좌표']]
df_morning = df_morning.rename(columns = {'X좌표': '경도', 'Y좌표': '위도'})
df_morning = df_morning.drop(index=25)

df_morning

,ARS_ID,출근시간_수요,정류소명,경도,위도
0,12010,4351,은평뉴타운10단지,126.913730,37.643407
1,12014,6777,박석고개.신도고등학교,126.919017,37.631088
2,12111,7726,구파발역2번출구,126.918989,37.636768
3,12112,7100,구파발역.롯데몰,126.918577,37.637248
4,12192,2904,신도중학교,126.931529,37.633348
5,12197,2767,폭포동.은평노인종합복지관,126.931007,37.631383
6,12351,5883,제각말5단지.은평뉴타운도서관,126.934438,37.636736
7,12430,3513,진관중고교.약수사앞,126.923090,37.643408
8,12432,6158,푸르지오621동앞,126.930479,37.648435
9,12434,3247,생태공원앞.구립상림도서관,126.928970,37.644075


# 출근 클러스터링

In [28]:
# 클러스터링에 사용할 데이터 (위도와 경도)
coordinates_출근 = df_morning[['위도', '경도']]

# K-Means 클러스터링 수행
num_clusters_출근 = 4  # 원하는 클러스터 개수
kmeans_출근 = KMeans(n_clusters=num_clusters_출근, random_state=42, n_init=10)
df_morning['클러스터'] = kmeans_출근.fit_predict(coordinates_출근)

# 결과 출력
df_morning

,ARS_ID,출근시간_수요,정류소명,경도,위도,클러스터
0,12010,4351,은평뉴타운10단지,126.913730,37.643407,0
1,12014,6777,박석고개.신도고등학교,126.919017,37.631088,0
2,12111,7726,구파발역2번출구,126.918989,37.636768,0
3,12112,7100,구파발역.롯데몰,126.918577,37.637248,0
4,12192,2904,신도중학교,126.931529,37.633348,0
5,12197,2767,폭포동.은평노인종합복지관,126.931007,37.631383,0
6,12351,5883,제각말5단지.은평뉴타운도서관,126.934438,37.636736,0
7,12430,3513,진관중고교.약수사앞,126.923090,37.643408,0
8,12432,6158,푸르지오621동앞,126.930479,37.648435,0
9,12434,3247,생태공원앞.구립상림도서관,126.928970,37.644075,0


# 출근 클러스터링 시각화

In [29]:
import folium

# Folium 지도 생성
map_center = [df_morning['위도'].mean(), df_morning['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 클러스터링 결과를 지도에 추가
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen', 'pink', 'cadetblue']
colors_출근 = colors[:num_clusters_출근]
for _, row in df_morning.iterrows():
    cluster = row['클러스터']
    color = colors_출근[cluster % len(colors_출근)]  # 클러스터별 색상 할당
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Station: {row['ARS_ID']}, Cluster: {cluster}"
    ).add_to(m)

# 결과 출력
m

In [30]:
# 결과 저장
m.save('clustering_result_map_출근.html')

# 클러스터 시작점 끝점 함수 정의

In [31]:
def find_closest_point_between_clusters(cluster1, cluster2):
    """
    두 클러스터 간에서 가장 가까운 두 점을 반환.
    cluster1에서 선택된 점은 반드시 cluster1에 속하며,
    cluster2에서 선택된 점은 반드시 cluster2에 속합니다.
    """
    cluster1_coords = cluster1[['위도', '경도']].values
    cluster2_coords = cluster2[['위도', '경도']].values
    distances = cdist(cluster1_coords, cluster2_coords, metric='euclidean')
    i, j = np.unravel_index(np.argmin(distances), distances.shape)
    return cluster1.iloc[i], cluster2.iloc[j]


def find_farthest_point(cluster_df, reference_point):
    """클러스터 내에서 reference_point와 가장 먼 점 찾기"""
    # 수치형 변환 (명시적으로 float64로 변환)
    reference_point = np.array(reference_point, dtype=np.float64).reshape(1, -1)
    distances = cdist(reference_point, cluster_df[['위도', '경도']].values, metric='euclidean')
    farthest_idx = np.argmax(distances)
    return cluster_df.iloc[farthest_idx]

def find_closest_point(cluster_df, reference_point):
    """클러스터 내에서 reference_point와 가장 가까운 점 찾기"""
    # reference_point가 객체가 아닌 수치형인지 확인 후 변환
    reference_point = np.array(reference_point, dtype=np.float64).reshape(1, -1)
    distances = cdist(reference_point, cluster_df[['위도', '경도']].values, metric='euclidean')
    closest_idx = np.argmin(distances)
    return cluster_df.iloc[closest_idx]

# 출근 클러스터 순서 결정 (TSP)

In [32]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from itertools import permutations

# 출근 클러스터 순서 결정
# 3. 클러스터 간 최적 순서 결정 (TSP 방식)
cluster_centroids_출근 = pd.DataFrame(kmeans_출근.cluster_centers_, columns=['위도', '경도'])
cluster_centroids_출근['클러스터'] = range(num_clusters_출근)

def calculate_tsp_route(locations):
    """단순 TSP로 최단 경로 계산"""
    coords = locations[['위도', '경도']].values
    min_distance = float('inf')
    best_route = None
    for perm in permutations(range(len(coords))):
        dist = 0
        for i in range(len(perm) - 1):
            dist += np.linalg.norm(coords[perm[i]] - coords[perm[i + 1]])
        if dist < min_distance:
            min_distance = dist
            best_route = perm
    return best_route, min_distance

# 클러스터 중심들로 최단 경로 계산
route_출근, route_distance_출근 = calculate_tsp_route(cluster_centroids_출근)

In [33]:
route_출근 = tuple([3, 2, 0, 1])
route_출근

(3, 2, 0, 1)

# 출근 클러스터 순서 기반 시작점 끝점 정의

In [34]:
# 클러스터 경계 최적화 (출근)
final_route_출근 = []
for i in range(len(route_출근)):
    current_cluster = df_morning[df_morning['클러스터'] == route_출근[i]]

    if i == 0:  # 첫 클러스터
        # 첫 클러스터의 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_morning[df_morning['클러스터'] == route_출근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

        # 첫 클러스터의 시작점: 끝점과 가장 먼 점
        start_point = find_farthest_point(current_cluster, end_point[['위도', '경도']].values)

    elif i == len(route_출근) - 1:  # 마지막 클러스터
        # 마지막 클러스터의 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route_출근[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 마지막 클러스터의 끝점: 시작점과 가장 먼 점
        end_point = find_farthest_point(current_cluster, start_point[['위도', '경도']].values)

    else:  # 중간 클러스터
        # 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route_출근[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_morning[df_morning['클러스터'] == route_출근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

    final_route_출근.append({'클러스터': route_출근[i], '시작점': start_point, '끝점': end_point})

# 결과 출력
print("출근시간 최적의 클러스터 간 순서:")
print(" -> ".join([f"클러스터 {cluster_centroids_출근.iloc[i]['클러스터']}" for i in route_출근]))
print("\n클러스터 경계 정류장:")
for item in final_route_출근:
    print(f"클러스터 {item['클러스터']}: 시작점 - {item['시작점']['ARS_ID']}, 끝점 - {item['끝점']['ARS_ID']}")

출근시간 최적의 클러스터 간 순서:
클러스터 3.0 -> 클러스터 2.0 -> 클러스터 0.0 -> 클러스터 1.0

클러스터 경계 정류장:
클러스터 3: 시작점 - 14016, 끝점 - 14016
클러스터 2: 시작점 - 12381, 끝점 - 12016
클러스터 0: 시작점 - 12014, 끝점 - 12197
클러스터 1: 시작점 - 13036, 끝점 - 2006


# 출근 유전 알고리즘 함수 정의

In [ ]:
from geopy.distance import geodesic

# 군집별로 정류장 좌표를 분리하고, 최대 거리를 계산
clusters_출근 = df_morning['클러스터'].unique()

cluster_max_distances_출근 = {}

for cluster in clusters_출근:
    # 해당 군집의 좌표 추출
    cluster_coords = df_morning[df_morning['클러스터'] == cluster][['위도', '경도']].values
    max_distance = 0
    # 군집 내 정류장 간 거리 계산
    for i in range(len(cluster_coords)):
        for j in range(i + 1, len(cluster_coords)):
            dist = geodesic(cluster_coords[i], cluster_coords[j]).meters
            if dist > max_distance:
                max_distance = dist
    cluster_max_distances_출근[cluster] = max_distance

cluster_max_distances_출근

In [21]:
import numpy as np
import random
import osmnx as ox
import networkx as nx
from geopy.distance import geodesic

# 정류장 평균 좌표
center_lat_출근 = df_morning['위도'].mean()
center_lon_출근 = df_morning['경도'].mean()

# 정류장 중심으로 네트워크 제한
G = ox.graph_from_point((center_lat_출근, center_lon_출근), dist=5000, network_type="drive")

# Parameters for GA
mutation_rate = 0.2
generations = 50
population_size = 20
distance_threshold = 400  # meters
stop_time = 30  # seconds
time_per_passenger = 3  # seconds

# 도로 거리 계산 함수
def calculate_road_distance(coord1, coord2):
    # 좌표를 노드로 변환
    node1 = ox.nearest_nodes(G, X=coord1[1], Y=coord1[0])
    node2 = ox.nearest_nodes(G, X=coord2[1], Y=coord2[0])

    # 최단 경로 거리 계산
    try:
        distance = nx.shortest_path_length(G, node1, node2, weight="length")
    except nx.NetworkXNoPath:
        # 경로가 없을 경우 유클리디안 거리 사용 (백업)
        distance = geodesic(coord1, coord2).meters

    return distance

# Function to calculate total route distance
def calculate_route_distance(route, coords):
    distance = 0
    for i in range(len(route) - 1):
        coord1 = coords[route[i]]
        coord2 = coords[route[i + 1]]
        distance += calculate_road_distance(coord1, coord2)
    return distance

# Function to calculate stop time
def calculate_stop_time(route, demand):
    return sum(stop_time + time_per_passenger * demand[i] for i in route)

# Generate initial population
def initialize_population(n_stations, population_size):
    return [random.sample(range(n_stations), n_stations) for _ in range(population_size)]

# Selection
def select_parents(population, fitness_scores):
    sorted_indices = np.argsort(fitness_scores)
    if len(sorted_indices) >= 2:
        indices = sorted_indices[-2:]  # Top 2
    else:
        indices = sorted_indices
    return [population[i] for i in indices]

# Ordered Crossover
def ordered_crossover(parent1, parent2, start_point, end_point):
    size = len(parent1)
    if size <= 3:
        return parent1  # 중간 정류장이 없거나 하나인 경우 부모 반환
    child = [None] * size
    a, b = sorted(random.sample(range(1, size - 1), 2))
    child[a:b] = parent1[a:b]

    # Fix range concatenation issue
    p2_index = b
    remaining_indices = list(range(b, size - 1)) + list(range(1, b))
    for idx in remaining_indices:
        if parent2[idx] not in child:
            child[p2_index % (size - 1)] = parent2[idx]
            p2_index += 1

    child[0] = start_point
    child[-1] = end_point
    # return child
    return [x for x in child if x is not None]  # Ensure no `None` values


# Mutation
def mutate(route, start_point, end_point):
    if random.random() < mutation_rate and len(route) > 3:
        idx = range(1, len(route) - 1)
        i, j = random.sample(idx, 2)
        route[i], route[j] = route[j], route[i]
    return route

# Genetic Algorithm
def genetic_algorithm(coords, demand, generations, population_size, start_point, end_point):
    n_stations = len(coords)
    middle_stations = [i for i in range(n_stations) if i != start_point and i != end_point]

    # 경로가 시작점과 끝점만으로 구성된 경우 처리
    if len(middle_stations) == 0:
        best_route = [start_point, end_point]
        best_fitness = calculate_route_fitness(best_route, coords, demand)
        return best_route, best_fitness
    elif len(middle_stations) == 1:
        best_route = [start_point, middle_stations[0], end_point]
        best_fitness = calculate_route_fitness(best_route, coords, demand)
        return best_route, best_fitness

    # 초기 population 생성
    population = [random.sample(middle_stations, len(middle_stations)) for _ in range(population_size)]
    population = [[start_point] + p + [end_point] for p in population]

    for generation in range(generations):

        # 진행 상황 출력
        print(f"Generation {generation + 1}/{generations} is running...")

        # 각 경로에 대해 적합도 계산 및 경로 최적화
        fitness_scores = []
        for route in population:
            fitness = calculate_route_fitness(route, coords, demand)
            fitness_scores.append(fitness)

        # 새로운 population 생성
        new_population = []
        for _ in range(population_size // 2):
            parents = select_parents(population, fitness_scores)
            child1 = ordered_crossover(parents[0], parents[1], start_point, end_point)
            child2 = ordered_crossover(parents[1], parents[0], start_point, end_point)
            child1 = mutate(child1, start_point, end_point)
            child2 = mutate(child2, start_point, end_point)
            new_population.extend([child1, child2])

        # population 업데이트
        population = new_population

    # 최적 경로와 적합도 반환
    best_index = np.argmax(fitness_scores)
    best_route = population[best_index]
    best_fitness = fitness_scores[best_index]
    return best_route, best_fitness


def calculate_route_fitness(route, coords, demand):
    # 노드별 값 저장
    demand_array = []
    distance_array = []
    time_array = []
    penalty_array = []

    demand_weight = 1.0
    distance_weight = 1.0
    time_weight = 1.0
    penalty_weight = 30.0

    for i in range(len(route)):
        # 현재 노드의 수요
        node_demand = demand[route[i]]

        # 다음 노드와의 거리 계산 (마지막 노드는 제외)
        if i < len(route) - 1:
            node_distance = calculate_road_distance(coords[route[i]], coords[route[i + 1]])
            # 거리 기반 패널티 계산
            if node_distance < distance_threshold:
                penalty = (distance_threshold - node_distance) * penalty_weight  # 초과된 거리만큼 패널티
            else:
                penalty = 0
        else:
            node_distance = 0
            penalty = 0

        # 패널티가 수요보다 크면 정류장 제외
        if penalty > node_demand and i < len(route) - 1:  # 마지막 노드는 제외
            continue  # 현재 노드를 무시

        # 현재 노드의 정차 시간
        node_stop_time = stop_time + time_per_passenger * node_demand

        # 배열에 추가
        demand_array.append(node_demand)
        distance_array.append(node_distance)
        time_array.append(node_stop_time)
        penalty_array.append(penalty)

    # numpy 배열로 변환
    demand_array = np.array(demand_array)
    distance_array = np.array(distance_array)
    time_array = np.array(time_array)
    penalty_array = np.array(penalty_array)

    # 실시간 스케일링
    if len(demand_array) > 1:
        scaled_demand = (demand_array - demand_array.min()) / (demand_array.max() - demand_array.min())
    else:
        scaled_demand = demand_array

    if len(distance_array) > 1:
        scaled_distance = (distance_array - distance_array.min()) / (distance_array.max() - distance_array.min())
    else:
        scaled_distance = distance_array

    if len(time_array) > 1:
        scaled_time = (time_array - time_array.min()) / (time_array.max() - time_array.min())
    else:
        scaled_time = time_array

    # 패널티 스케일링 수정 (값이 모두 0인 경우 처리)
    if len(penalty_array) > 0 and np.any(penalty_array != 0):
        scaled_penalty = (penalty_array - penalty_array.min()) / (penalty_array.max() - penalty_array.min())
    else:
        scaled_penalty = penalty_array * 0  # 패널티가 모두 0인 경우

    # 총합 계산
    total_demand = np.sum(scaled_demand)
    total_distance = np.sum(scaled_distance)
    total_stop_time = np.sum(scaled_time)
    total_penalty = np.sum(scaled_penalty)


    # 적합도 계산

    fitness_value = (demand_weight * total_demand) - \
                    (distance_weight * total_distance + time_weight * total_stop_time + penalty_weight * total_penalty)

    print(f"Fitness Value: {fitness_value}")
    return fitness_value



def execute_ga_for_cluster_출근(cluster_data, distance_threshold, start_point, end_point):
    coords = cluster_data[['위도', '경도']].values
    demand = cluster_data['출근시간_수요'].values

    print("\n[Debug] Inside execute_ga_for_cluster_출근")
    print(f"Coords: {coords}")
    print(f"Demand: {demand}")
    print(f"Start Point: {start_point}, End Point: {end_point}")

    # Execute GA with fixed start and end points
    best_route, best_fitness = genetic_algorithm(coords, demand, generations, population_size, start_point, end_point)

    # Map indices to ARS_IDs
    route_ids = [cluster_data.iloc[i]['ARS_ID'] for i in best_route]
    print(f"[Cluster Result] Best Route: {best_route}, Fitness: {best_fitness}")
    return route_ids, best_fitness

# 출근 유전 알고리즘 실행

In [22]:
# Apply GA to each cluster with user-defined start and end points
final_routes_출근 = {}

start_ends_출근 = {
    3: {'start': '12432', 'end': '12111'},
    2: {'start': '12014', 'end': '12014'},
    0: {'start': '12016', 'end': '12381'},
    1: {'start': '13036', 'end': '2006'}
}

# Ensure ARS_ID and start_ends values are strings
df_morning['ARS_ID'] = df_morning['ARS_ID'].astype(str)
start_ends_출근 = {k: {'start': str(v['start']), 'end': str(v['end'])} for k, v in start_ends_출근.items()}


for cluster_id in route_출근:
    # Filter current cluster data
    current_cluster = df_morning[df_morning['클러스터'] == cluster_id].reset_index(drop=True)

    # Debug: Check cluster ARS_IDs
    print(f"Cluster {cluster_id} ARS_IDs: {current_cluster['ARS_ID'].tolist()}")

    # Find start and end points based on user-defined `start_ends`
    start_matches = current_cluster[current_cluster['ARS_ID'] == start_ends_출근[cluster_id]['start']]
    end_matches = current_cluster[current_cluster['ARS_ID'] == start_ends_출근[cluster_id]['end']]

    # Ensure start and end points are valid
    if start_matches.empty:
        print(f"Warning: Start point {start_ends_출근[cluster_id]['start']} not found in cluster {cluster_id}. Using first point.")
        start_point = 0
    else:
        start_point = start_matches.index[0]

    if end_matches.empty:
        print(f"Warning: End point {start_ends_출근[cluster_id]['end']} not found in cluster {cluster_id}. Using last point.")
        end_point = len(current_cluster) - 1
    else:
        end_point = end_matches.index[0]

    # Debug: Check assigned start and end points
    print(f"Cluster {cluster_id}: Start - {current_cluster.iloc[start_point]['ARS_ID']}, "
          f"End - {current_cluster.iloc[end_point]['ARS_ID']}")

    # Execute genetic algorithm for the cluster
    try:
        route_ids, fitness_score = execute_ga_for_cluster_출근(current_cluster, distance_threshold, start_point, end_point)
        final_routes_출근[cluster_id] = {'route': route_ids, 'fitness': fitness_score}
    except Exception as e:
        print(f"Error in cluster {cluster_id}: {e}")
        continue

# Display final results
for cluster_id, details in final_routes_출근.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Route: {' -> '.join(map(str, details['route']))}")
    print(f"  Fitness Score: {details['fitness']}\n")

Cluster 3 ARS_IDs: ['14016']
Cluster 3: Start - 14016, End - 14016

[Debug] Inside execute_ga_for_cluster_출근
Coords: [[ 37.55732564 126.9243163 ]]
Demand: [26520]
Start Point: 0, End Point: 0
Fitness Value: nan
[Cluster Result] Best Route: [0, 0], Fitness: nan
Cluster 2 ARS_IDs: ['12016', '12018', '12019', '12184', '12201', '12202', '12381']
Cluster 2: Start - 12016, End - 12381

[Debug] Inside execute_ga_for_cluster_출근
Coords: [[ 37.62528259 126.9199961 ]
 [ 37.61932256 126.9209337 ]
 [ 37.61560685 126.9250897 ]
 [ 37.61782072 126.9199962 ]
 [ 37.62051965 126.9243797 ]
 [ 37.61959665 126.9225341 ]
 [ 37.60087211 126.9342261 ]]
Demand: [ 5066 12609  5466 20715  6312 14539 36335]
Start Point: 0, End Point: 6
Generation 1/50 is running...


<ipython-input-21-b7844cfdd8e0>:193: RuntimeWarning: invalid value encountered in divide
  scaled_demand = (demand_array - demand_array.min()) / (demand_array.max() - demand_array.min())
<ipython-input-21-b7844cfdd8e0>:198: RuntimeWarning: invalid value encountered in divide
  scaled_distance = (distance_array - distance_array.min()) / (distance_array.max() - distance_array.min())
<ipython-input-21-b7844cfdd8e0>:203: RuntimeWarning: invalid value encountered in divide
  scaled_time = (time_array - time_array.min()) / (time_array.max() - time_array.min())


Fitness Value: -38.74021721917813
Fitness Value: -38.39803689177274
Fitness Value: -60.89323039045285
Fitness Value: -51.56411611317854
Fitness Value: -32.548379389754025
Fitness Value: -61.134210330214806
Fitness Value: -32.214597104602895
Fitness Value: -56.617713927374865
Fitness Value: -32.310767859704384
Fitness Value: -61.18052262947395
Fitness Value: -32.443833087093644
Fitness Value: -32.61667540005347
Fitness Value: -32.443833087093644
Fitness Value: -32.267257007382305
Fitness Value: -2.262001116492839
Fitness Value: -32.373895802643226
Fitness Value: -31.975470943125714
Fitness Value: -38.44114958568952
Fitness Value: -32.267257007382305
Fitness Value: -2.335731832135643
Generation 2/50 is running...
Fitness Value: -2.0894204132690852
Fitness Value: -2.099608578498907
Fitness Value: -32.1608168047717
Fitness Value: -32.13920367171445
Fitness Value: -1.9688132859385497
Fitness Value: -2.262001116492839
Fitness Value: -32.0844801048545
Fitness Value: -32.13920367171445
Fitness

# 출근 클러스터 별 유전 결과 시각화

In [23]:
import osmnx as ox
import networkx as nx
import folium
from geopy.distance import geodesic

# 지도 초기화
map_clusters_출근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 색상 리스트 (군집별 구분용)
colors = ['blue', 'green', 'purple', 'orange', 'darkred']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_출근.keys())}

# 도로 네트워크 가져오기 (모든 정류장이 포함되도록 범위 확장)
center_lat_출근 = df_morning['위도'].mean()
center_lon_출근 = df_morning['경도'].mean()
G = ox.graph_from_point((center_lat_출근, center_lon_출근), dist=10000, network_type='drive')

# 군집별 경로 시각화
for cluster_id, details in final_routes_출근.items():
    route = details['route']
    cluster_data = df_morning[df_morning['ARS_ID'].isin(route)]

    # 경로가 없는 경우 스킵
    if cluster_data.empty:
        print(f"Cluster {cluster_id}: No valid stops found.")
        continue

    # 정류장 순서를 클러스터 내 경로 순서에 맞게 정렬
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')

    coords = cluster_data[['위도', '경도']].values

    # 도로 경로 생성
    road_route = []
    for i in range(len(coords) - 1):
        # 시작점과 끝점 노드 찾기
        start_node = ox.distance.nearest_nodes(G, X=coords[i][1], Y=coords[i][0])
        end_node = ox.distance.nearest_nodes(G, X=coords[i + 1][1], Y=coords[i + 1][0])

        # 최단 경로 계산
        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            road_route.extend(shortest_path)
        except nx.NetworkXNoPath:
            print(f"Cluster {cluster_id}: No path between {coords[i]} and {coords[i + 1]}. Using direct line.")
            # 경로가 없으면 직선 연결
            road_route.append((coords[i][0], coords[i][1]))
            road_route.append((coords[i + 1][0], coords[i + 1][1]))

    # 중복된 노드 제거
    road_route = list(dict.fromkeys(road_route))

    # 도로 경로 좌표 추출
    road_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in road_route]

    # 도로 경로 추가
    cluster_color = cluster_colors[cluster_id]
    folium.PolyLine(
        locations=road_coords,
        color=cluster_color,
        weight=5,
        opacity=0.8,
        tooltip=f"Cluster {cluster_id} Road Route"
    ).add_to(map_clusters_출근)

    # 정류장 마커 추가
    for _, row in cluster_data.iterrows():
        icon_color = 'red' if row['순서'] == 0 or row['순서'] == len(route) - 1 else cluster_color
        folium.Marker(
            location=[row['위도'], row['경도']],
            tooltip=f"Cluster {cluster_id} - Stop {row['순서'] + 1} (ARS_ID: {row['ARS_ID']})",
            icon=folium.Icon(color=icon_color, icon="info-sign")
        ).add_to(map_clusters_출근)

# 지도 저장
map_clusters_출근.save("optimized_cluster_routes_map_출근.html")
print("지도 저장 완료: 'optimized_cluster_routes_map_출근.html'")
map_clusters_출근

<ipython-input-23-a3b5096dde4e>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: route.index(x))


ValueError: Locations is empty.

# 출근 최종 노선 생성 (클러스터 연결) 및 시각화

In [24]:
import osmnx as ox
import networkx as nx
import folium
from folium.features import DivIcon

# 지도 초기화
final_map_출근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 도로 네트워크 가져오기
center_lat_출근 = df_morning['위도'].mean()
center_lon_출근 = df_morning['경도'].mean()
G = ox.graph_from_point((center_lat_출근, center_lon_출근), dist=10000, network_type='drive')

# 색상 설정
colors = ['blue', 'green', 'purple', 'orange', 'cyan']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_출근.keys())}

# 최종 노선 경로 저장
final_route_coords_출근 = []  # 도로 경로 포함 전체 경로
all_stop_coords = []  # 정류장 마커 표시용

# 클러스터 순서대로 경로 연결
clusters_order = list(final_routes_출근.keys())  # 클러스터 순서
for i, cluster_id in enumerate(clusters_order):
    # 현재 클러스터의 경로와 좌표 가져오기
    cluster_route = final_routes_출근[cluster_id]['route']
    cluster_data = df_morning[df_morning['ARS_ID'].isin(cluster_route)].copy()

    # 클러스터 내 정류장 순서 유지
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: cluster_route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')
    cluster_coords = cluster_data[['위도', '경도']].values

    # 클러스터 내 경로를 지도에 추가 (도로 경로로 표시)
    for j in range(len(cluster_coords) - 1):
        start_node = ox.distance.nearest_nodes(G, X=cluster_coords[j][1], Y=cluster_coords[j][0])
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[j + 1][1], Y=cluster_coords[j + 1][0])

        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color=cluster_colors[cluster_id],
                weight=5,
                opacity=0.8,
                tooltip=f"Cluster {cluster_id} Route"
            ).add_to(final_map_출근)
        except nx.NetworkXNoPath:
            print(f"No path between stops {j} and {j + 1} in Cluster {cluster_id}.")
            continue

    # 정류장 마커 추가
    all_stop_coords.extend(cluster_coords)

    # 클러스터 간 연결 (현재 클러스터 끝점과 다음 클러스터 시작점)
    if i < len(clusters_order) - 1:
        next_cluster_id = clusters_order[i + 1]
        next_cluster_route = final_routes_출근[next_cluster_id]['route']
        next_cluster_data = df_morning[df_morning['ARS_ID'].isin(next_cluster_route)].copy()
        next_cluster_data['순서'] = next_cluster_data['ARS_ID'].apply(lambda x: next_cluster_route.index(x))
        next_cluster_data = next_cluster_data.sort_values(by='순서')
        next_cluster_coords = next_cluster_data[['위도', '경도']].values

        # 현재 클러스터 끝점과 다음 클러스터 시작점 연결
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[-1][1], Y=cluster_coords[-1][0])
        start_node = ox.distance.nearest_nodes(G, X=next_cluster_coords[0][1], Y=next_cluster_coords[0][0])

        try:
            shortest_path = nx.shortest_path(G, end_node, start_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 간 연결 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color='black',
                weight=5,
                opacity=0.6,
                tooltip=f"Connection between Cluster {cluster_id} and {next_cluster_id}"
            ).add_to(final_map_출근)
        except nx.NetworkXNoPath:
            print(f"No path between Cluster {cluster_id} and Cluster {next_cluster_id}.")
            continue

# 정류장 마커 추가 (모든 정류장 표시)
for i, coord in enumerate(all_stop_coords):
    marker_color = 'red' if i == 0 or i == len(all_stop_coords) - 1 else 'blue'
    folium.CircleMarker(
        location=coord,
        radius=6,
        color=marker_color,
        fill=True,
        fill_opacity=0.7,
        tooltip=f"Stop {i + 1}"
    ).add_to(final_map_출근)

# 지도 저장
final_map_출근.save("final_cluster_route_map_출근.html")
print("지도 저장 완료: 'final_cluster_route_map_출근.html'")
final_map_출근

지도 저장 완료: 'final_cluster_route_map_출근.html'


# 출근 최종 노선 시각화 포인트 수정

In [25]:
import osmnx as ox
import networkx as nx
import folium
from folium.features import DivIcon

# 지도 초기화
final_map_출근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 도로 네트워크 가져오기
center_lat_출근 = df_morning['위도'].mean()
center_lon_출근 = df_morning['경도'].mean()
G = ox.graph_from_point((center_lat_출근, center_lon_출근), dist=10000, network_type='drive')

# 정류장 순서 저장
stop_order = 1  # 전체 정류장 순서 시작
total_first_stop = True  # 전체 첫 정류장 플래그
total_last_stop_coords = None  # 전체 마지막 정류장 좌표 저장

# 클러스터 순서대로 경로 연결
clusters_order = list(final_routes_출근.keys())  # 클러스터 순서
for i, cluster_id in enumerate(clusters_order):
    # 현재 클러스터의 경로와 좌표 가져오기
    cluster_route = final_routes_출근[cluster_id]['route']
    cluster_data = df_morning[df_morning['ARS_ID'].isin(cluster_route)].copy()

    # 클러스터 내 정류장 순서 유지
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: cluster_route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')
    cluster_coords = cluster_data[['위도', '경도']].values

    # 클러스터 내 경로를 지도에 추가 (도로 경로로 표시)
    for j in range(len(cluster_coords) - 1):
        start_node = ox.distance.nearest_nodes(G, X=cluster_coords[j][1], Y=cluster_coords[j][0])
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[j + 1][1], Y=cluster_coords[j + 1][0])

        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color=cluster_colors[cluster_id],
                weight=5,
                opacity=0.8,
                tooltip=f"Cluster {cluster_id} Route"
            ).add_to(final_map_출근)
        except nx.NetworkXNoPath:
            print(f"No path between stops {j} and {j + 1} in Cluster {cluster_id}.")
            continue

    # 정류장 마커 추가 (전체 첫 정류장과 마지막 정류장 처리)
    for idx, coord in enumerate(cluster_coords):
        if total_first_stop:  # 전체 첫 정류장
            marker_color = 'red'
            total_first_stop = False
        elif i == len(clusters_order) - 1 and idx == len(cluster_coords) - 1:  # 전체 마지막 정류장
            marker_color = 'red'
        else:  # 중간 정류장
            marker_color = 'blue'

        # 마커 추가 (포인트 크기 증가)
        folium.CircleMarker(
            location=coord,
            radius=12,  # 마커 크기 증가
            color=marker_color,
            fill=True,
            fill_opacity=0.7
        ).add_to(final_map_출근)

        # 정류장 순서 텍스트 추가
        folium.map.Marker(
            location=coord,
            icon=DivIcon(
                icon_size=(200, 36),
                icon_anchor=(12, 12),  # 텍스트 중심 위치
                html=(f'<div style="font-size: 16px; color: white; font-weight: bold; '
                      f'text-shadow: -1px -1px 0 black, 1px -1px 0 black, -1px 1px 0 black, 1px 1px 0 black;">'
                      f'{stop_order}</div>')
            )
        ).add_to(final_map_출근)

        stop_order += 1

    # 다음 클러스터와 연결
    if i < len(clusters_order) - 1:
        next_cluster_id = clusters_order[i + 1]
        next_cluster_route = final_routes_출근[next_cluster_id]['route']
        next_cluster_data = df_morning[df_morning['ARS_ID'].isin(next_cluster_route)].copy()
        next_cluster_data['순서'] = next_cluster_data['ARS_ID'].apply(lambda x: next_cluster_route.index(x))
        next_cluster_data = next_cluster_data.sort_values(by='순서')
        next_cluster_coords = next_cluster_data[['위도', '경도']].values

        # 현재 클러스터 끝점과 다음 클러스터 시작점 연결
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[-1][1], Y=cluster_coords[-1][0])
        start_node = ox.distance.nearest_nodes(G, X=next_cluster_coords[0][1], Y=next_cluster_coords[0][0])

        try:
            shortest_path = nx.shortest_path(G, end_node, start_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 간 연결 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color='black',
                weight=5,
                opacity=0.6,
                tooltip=f"Connection between Cluster {cluster_id} and {next_cluster_id}"
            ).add_to(final_map_출근)
        except nx.NetworkXNoPath:
            print(f"No path between Cluster {cluster_id} and Cluster {next_cluster_id}.")
            continue

# 지도 저장
final_map_출근.save("final_route_map_출근.html")
print("지도 저장 완료: 'final_route_map_출근.html'")
final_map_출근

지도 저장 완료: 'final_route_map_출근.html'


# 퇴근 데이터 불러오기

In [35]:
df_evening = pd.read_csv('/content/drive/MyDrive/퇴근_정류장_좌표_수요.csv', encoding='CP949')
df_evening = df_evening[['ARS_ID', '퇴근시간_수요', '정류소명', 'X좌표', 'Y좌표']]
df_evening = df_evening.rename(columns = {'X좌표': '경도', 'Y좌표': '위도'})

df_evening

,ARS_ID,퇴근시간_수요,정류소명,경도,위도
0,22008,20754,래미안아파트.파이낸셜뉴스,127.030987,37.491087
1,22010,23574,신분당선강남역,127.029112,37.494993
2,22012,36951,지하철2호선강남역,127.026370,37.500785
3,23275,3201,선정릉역.무형문화재전수회관,127.044094,37.509234
4,23276,3414,선릉역,127.048176,37.505884
5,23277,8697,서울상록회관.한국기술센터,127.044057,37.503232
6,23278,5829,KT강남지사,127.041979,37.504755
7,23280,6307,아가방빌딩.하나은행,127.038771,37.501641
8,23281,7533,역삼역7번출구.GS타워,127.036766,37.501505
9,23282,4990,역삼역6번출구,127.036387,37.501541


# 퇴근 클러스터링

In [36]:
# 클러스터링에 사용할 데이터 (위도와 경도)
coordinates_퇴근 = df_evening[['위도', '경도']]

# K-Means 클러스터링 수행
num_clusters_퇴근 = 5  # 원하는 클러스터 개수
kmeans_퇴근 = KMeans(n_clusters=num_clusters_퇴근, random_state=42, n_init=10)
df_evening['클러스터'] = kmeans_퇴근.fit_predict(coordinates_퇴근)

# 결과 출력
df_evening

,ARS_ID,퇴근시간_수요,정류소명,경도,위도,클러스터
0,22008,20754,래미안아파트.파이낸셜뉴스,127.030987,37.491087,4
1,22010,23574,신분당선강남역,127.029112,37.494993,4
2,22012,36951,지하철2호선강남역,127.026370,37.500785,4
3,23275,3201,선정릉역.무형문화재전수회관,127.044094,37.509234,0
4,23276,3414,선릉역,127.048176,37.505884,0
5,23277,8697,서울상록회관.한국기술센터,127.044057,37.503232,0
6,23278,5829,KT강남지사,127.041979,37.504755,0
7,23280,6307,아가방빌딩.하나은행,127.038771,37.501641,0
8,23281,7533,역삼역7번출구.GS타워,127.036766,37.501505,0
9,23282,4990,역삼역6번출구,127.036387,37.501541,0


In [37]:
# 결과 저장
m.save('clustering_result_map_퇴근.html')

# 퇴근 클러스터링 시각화

In [52]:
import folium

# Folium 지도 생성
map_center = [df_evening['위도'].mean(), df_evening['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 클러스터링 결과를 지도에 추가
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen', 'pink', 'cadetblue']
colors_퇴근 = colors[:num_clusters_퇴근]
for _, row in df_evening.iterrows():
    cluster = row['클러스터']
    color = colors[cluster % len(colors)]  # 클러스터별 색상 할당
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Station: {row['ARS_ID']}, Cluster: {cluster}"
    ).add_to(m)

# 결과 출력
m

# 퇴근 클러스터 순서 결정 (TSP)

In [53]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from itertools import permutations


# 3. 클러스터 간 최적 순서 결정 (TSP 방식)
cluster_centroids_퇴근 = pd.DataFrame(kmeans_퇴근.cluster_centers_, columns=['위도', '경도'])
cluster_centroids_퇴근['클러스터'] = range(num_clusters_퇴근)

def calculate_tsp_route(cluster_centroids):
    """단순 TSP로 최단 경로 계산"""
    coords = cluster_centroids[['위도', '경도']].values
    min_distance = float('inf')
    best_route = None
    for perm in permutations(range(len(coords))):
        dist = 0
        for i in range(len(perm) - 1):
            dist += np.linalg.norm(coords[perm[i]] - coords[perm[i + 1]])
        if dist < min_distance:
            min_distance = dist
            best_route = perm
    return best_route, min_distance

# 클러스터 중심들로 최단 경로 계산
route_퇴근, route_distance_퇴근 = calculate_tsp_route(cluster_centroids_퇴근)

In [40]:
route_퇴근

(1, 3, 4, 0, 2)

# 퇴근 클러스터 순서 기반 시작점 끝점 정의

In [41]:
# 클러스터 경계 최적화 (퇴근)
final_route_퇴근 = []
for i in range(len(route_퇴근)):
    current_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i]]

    if i == 0:  # 첫 클러스터
        # 첫 클러스터의 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

        # 첫 클러스터의 시작점: 끝점과 가장 먼 점
        start_point = find_farthest_point(current_cluster, end_point[['위도', '경도']].values)

    elif i == len(route_퇴근) - 1:  # 마지막 클러스터
        # 마지막 클러스터의 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route_퇴근[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 마지막 클러스터의 끝점: 시작점과 가장 먼 점
        end_point = find_farthest_point(current_cluster, start_point[['위도', '경도']].values)

    else:  # 중간 클러스터
        # 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route_퇴근[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

    final_route_퇴근.append({'클러스터': route_퇴근[i], '시작점': start_point, '끝점': end_point})

# 결과 출력
print("퇴근시간 최적의 클러스터 간 순서:")
print(" -> ".join([f"클러스터 {cluster_centroids_퇴근.iloc[i]['클러스터']}" for i in route_퇴근]))
print("\n클러스터 경계 정류장:")
for item in final_route_퇴근:
    print(f"클러스터 {item['클러스터']}: 시작점 - {item['시작점']['ARS_ID']}, 끝점 - {item['끝점']['ARS_ID']}")

퇴근시간 최적의 클러스터 간 순서:
클러스터 1.0 -> 클러스터 3.0 -> 클러스터 4.0 -> 클러스터 0.0 -> 클러스터 2.0

클러스터 경계 정류장:
클러스터 1: 시작점 - 3165, 끝점 - 3166
클러스터 3: 시작점 - 22015, 끝점 - 22014
클러스터 4: 시작점 - 23286, 끝점 - 23284
클러스터 0: 시작점 - 23283, 끝점 - 23283
클러스터 2: 시작점 - 22004, 끝점 - 22293


# 퇴근 유전 알고리즘 함수 정의

In [42]:
from geopy.distance import geodesic

# 군집별로 정류장 좌표를 분리하고, 최대 거리를 계산
clusters_퇴근 = df_evening['클러스터'].unique()

cluster_max_distances_퇴근 = {}

for cluster in clusters_퇴근:
    # 해당 군집의 좌표 추출
    cluster_coords = df_evening[df_evening['클러스터'] == cluster][['위도', '경도']].values
    max_distance = 0
    # 군집 내 정류장 간 거리 계산
    for i in range(len(cluster_coords)):
        for j in range(i + 1, len(cluster_coords)):
            dist = geodesic(cluster_coords[i], cluster_coords[j]).meters
            if dist > max_distance:
                max_distance = dist
    cluster_max_distances_퇴근[cluster] = max_distance

cluster_max_distances_퇴근

{4: 1332.8810448184634,
 0: 1333.7904402611812,
 2: 1389.4396895523375,
 3: 1644.5081999240542,
 1: 371.7311638848325}

In [43]:
import numpy as np
import random
import osmnx as ox
import networkx as nx
from geopy.distance import geodesic

# 정류장 평균 좌표
center_lat_퇴근 = df_evening['위도'].mean()
center_lon_퇴근 = df_evening['경도'].mean()

# 정류장 중심으로 네트워크 제한
G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=5000, network_type="drive")

# Parameters for GA
mutation_rate = 0.2
generations = 50
population_size = 20
distance_threshold = 400  # meters
stop_time = 30  # seconds
time_per_passenger = 3  # seconds

# 도로 거리 계산 함수
def calculate_road_distance(coord1, coord2):
    # 좌표를 노드로 변환
    node1 = ox.nearest_nodes(G, X=coord1[1], Y=coord1[0])
    node2 = ox.nearest_nodes(G, X=coord2[1], Y=coord2[0])

    # 최단 경로 거리 계산
    try:
        distance = nx.shortest_path_length(G, node1, node2, weight="length")
    except nx.NetworkXNoPath:
        # 경로가 없을 경우 유클리디안 거리 사용 (백업)
        distance = geodesic(coord1, coord2).meters

    return distance

# Function to calculate total route distance
def calculate_route_distance(route, coords):
    distance = 0
    for i in range(len(route) - 1):
        coord1 = coords[route[i]]
        coord2 = coords[route[i + 1]]
        distance += calculate_road_distance(coord1, coord2)
    return distance

# Function to calculate stop time
def calculate_stop_time(route, demand):
    return sum(stop_time + time_per_passenger * demand[i] for i in route)

# Generate initial population
def initialize_population(n_stations, population_size):
    return [random.sample(range(n_stations), n_stations) for _ in range(population_size)]

# Selection
def select_parents(population, fitness_scores):
    sorted_indices = np.argsort(fitness_scores)
    if len(sorted_indices) >= 2:
        indices = sorted_indices[-2:]  # Top 2
    else:
        indices = sorted_indices
    return [population[i] for i in indices]

# Ordered Crossover
def ordered_crossover(parent1, parent2, start_point, end_point):
    size = len(parent1)
    if size <= 3:
        return parent1  # 중간 정류장이 없거나 하나인 경우 부모 반환
    child = [None] * size
    a, b = sorted(random.sample(range(1, size - 1), 2))
    child[a:b] = parent1[a:b]

    # Fix range concatenation issue
    p2_index = b
    remaining_indices = list(range(b, size - 1)) + list(range(1, b))
    for idx in remaining_indices:
        if parent2[idx] not in child:
            child[p2_index % (size - 1)] = parent2[idx]
            p2_index += 1

    child[0] = start_point
    child[-1] = end_point
    # return child
    return [x for x in child if x is not None]  # Ensure no `None` values


# Mutation
def mutate(route, start_point, end_point):
    if random.random() < mutation_rate and len(route) > 3:
        idx = range(1, len(route) - 1)
        i, j = random.sample(idx, 2)
        route[i], route[j] = route[j], route[i]
    return route

# Genetic Algorithm
def genetic_algorithm(coords, demand, generations, population_size, start_point, end_point):
    n_stations = len(coords)
    middle_stations = [i for i in range(n_stations) if i != start_point and i != end_point]

    # 경로가 시작점과 끝점만으로 구성된 경우 처리
    if len(middle_stations) == 0:
        best_route = [start_point, end_point]
        best_fitness = calculate_route_fitness(best_route, coords, demand)
        return best_route, best_fitness
    elif len(middle_stations) == 1:
        best_route = [start_point, middle_stations[0], end_point]
        best_fitness = calculate_route_fitness(best_route, coords, demand)
        return best_route, best_fitness

    # 초기 population 생성
    population = [random.sample(middle_stations, len(middle_stations)) for _ in range(population_size)]
    population = [[start_point] + p + [end_point] for p in population]

    for generation in range(generations):

        # 진행 상황 출력
        print(f"Generation {generation + 1}/{generations} is running...")

        # 각 경로에 대해 적합도 계산 및 경로 최적화
        fitness_scores = []
        for route in population:
            fitness = calculate_route_fitness(route, coords, demand)
            fitness_scores.append(fitness)

        # 새로운 population 생성
        new_population = []
        for _ in range(population_size // 2):
            parents = select_parents(population, fitness_scores)
            child1 = ordered_crossover(parents[0], parents[1], start_point, end_point)
            child2 = ordered_crossover(parents[1], parents[0], start_point, end_point)
            child1 = mutate(child1, start_point, end_point)
            child2 = mutate(child2, start_point, end_point)
            new_population.extend([child1, child2])

        # population 업데이트
        population = new_population

    # 최적 경로와 적합도 반환
    best_index = np.argmax(fitness_scores)
    best_route = population[best_index]
    best_fitness = fitness_scores[best_index]
    return best_route, best_fitness


def calculate_route_fitness(route, coords, demand):
    # 노드별 값 저장
    demand_array = []
    distance_array = []
    time_array = []
    penalty_array = []

    demand_weight = 1.0
    distance_weight = 1.0
    time_weight = 1.0
    penalty_weight = 30.0

    for i in range(len(route)):
        # 현재 노드의 수요
        node_demand = demand[route[i]]

        # 다음 노드와의 거리 계산 (마지막 노드는 제외)
        if i < len(route) - 1:
            node_distance = calculate_road_distance(coords[route[i]], coords[route[i + 1]])
            # 거리 기반 패널티 계산
            if node_distance < distance_threshold:
                penalty = (distance_threshold - node_distance) * penalty_weight  # 초과된 거리만큼 패널티
            else:
                penalty = 0
        else:
            node_distance = 0
            penalty = 0

        # 패널티가 수요보다 크면 정류장 제외
        if penalty > node_demand and i < len(route) - 1:  # 마지막 노드는 제외
            continue  # 현재 노드를 무시

        # 현재 노드의 정차 시간
        node_stop_time = stop_time + time_per_passenger * node_demand

        # 배열에 추가
        demand_array.append(node_demand)
        distance_array.append(node_distance)
        time_array.append(node_stop_time)
        penalty_array.append(penalty)

    # numpy 배열로 변환
    demand_array = np.array(demand_array)
    distance_array = np.array(distance_array)
    time_array = np.array(time_array)
    penalty_array = np.array(penalty_array)

    # 실시간 스케일링
    if len(demand_array) > 1:
        scaled_demand = (demand_array - demand_array.min()) / (demand_array.max() - demand_array.min())
    else:
        scaled_demand = demand_array

    if len(distance_array) > 1:
        scaled_distance = (distance_array - distance_array.min()) / (distance_array.max() - distance_array.min())
    else:
        scaled_distance = distance_array

    if len(time_array) > 1:
        scaled_time = (time_array - time_array.min()) / (time_array.max() - time_array.min())
    else:
        scaled_time = time_array

    # 패널티 스케일링 수정 (값이 모두 0인 경우 처리)
    if len(penalty_array) > 0 and np.any(penalty_array != 0):
        scaled_penalty = (penalty_array - penalty_array.min()) / (penalty_array.max() - penalty_array.min())
    else:
        scaled_penalty = penalty_array * 0  # 패널티가 모두 0인 경우

    # 총합 계산
    total_demand = np.sum(scaled_demand)
    total_distance = np.sum(scaled_distance)
    total_stop_time = np.sum(scaled_time)
    total_penalty = np.sum(scaled_penalty)


    # 적합도 계산

    fitness_value = (demand_weight * total_demand) - \
                    (distance_weight * total_distance + time_weight * total_stop_time + penalty_weight * total_penalty)

    print(f"Fitness Value: {fitness_value}")
    return fitness_value



def execute_ga_for_cluster_퇴근(cluster_data, distance_threshold, start_point, end_point):
    coords = cluster_data[['위도', '경도']].values
    demand = cluster_data['퇴근시간_수요'].values

    print("\n[Debug] Inside execute_ga_for_cluster_퇴근")
    print(f"Coords: {coords}")
    print(f"Demand: {demand}")
    print(f"Start Point: {start_point}, End Point: {end_point}")

    # Execute GA with fixed start and end points
    best_route, best_fitness = genetic_algorithm(coords, demand, generations, population_size, start_point, end_point)

    # Map indices to ARS_IDs
    route_ids = [cluster_data.iloc[i]['ARS_ID'] for i in best_route]
    print(f"[Cluster Result] Best Route: {best_route}, Fitness: {best_fitness}")
    return route_ids, best_fitness

# 퇴근 유전 알고리즘 실행

In [45]:
# Apply GA to each cluster with user-defined start and end points
final_routes_퇴근 = {}

start_ends_퇴근 = {
    2: {'start': '22293', 'end': '22004'},
    0: {'start': '23281', 'end': '23282'},
    4: {'start': '23283', 'end': '23286'},
    1: {'start': '22014', 'end': '22015'},
    3: {'start': '3166', 'end': '3165'}
}

# Ensure ARS_ID and start_ends values are strings
df_evening['ARS_ID'] = df_evening['ARS_ID'].astype(str)
start_ends_퇴근 = {k: {'start': str(v['start']), 'end': str(v['end'])} for k, v in start_ends_퇴근.items()}


for cluster_id in route_퇴근:
    # Filter current cluster data
    current_cluster = df_evening[df_evening['클러스터'] == cluster_id].reset_index(drop=True)

    # Debug: Check cluster ARS_IDs
    print(f"Cluster {cluster_id} ARS_IDs: {current_cluster['ARS_ID'].tolist()}")

    # Find start and end points based on user-defined `start_ends`
    start_matches = current_cluster[current_cluster['ARS_ID'] == start_ends_퇴근[cluster_id]['start']]
    end_matches = current_cluster[current_cluster['ARS_ID'] == start_ends_퇴근[cluster_id]['end']]

    # Ensure start and end points are valid
    if start_matches.empty:
        print(f"Warning: Start point {start_ends_퇴근[cluster_id]['start']} not found in cluster {cluster_id}. Using first point.")
        start_point = 0
    else:
        start_point = start_matches.index[0]

    if end_matches.empty:
        print(f"Warning: End point {start_ends_퇴근[cluster_id]['end']} not found in cluster {cluster_id}. Using last point.")
        end_point = len(current_cluster) - 1
    else:
        end_point = end_matches.index[0]

    # Debug: Check assigned start and end points
    print(f"Cluster {cluster_id}: Start - {current_cluster.iloc[start_point]['ARS_ID']}, "
          f"End - {current_cluster.iloc[end_point]['ARS_ID']}")

    # Execute genetic algorithm for the cluster
    try:
        route_ids, fitness_score = execute_ga_for_cluster_퇴근(current_cluster, distance_threshold, start_point, end_point)
        final_routes_퇴근[cluster_id] = {'route': route_ids, 'fitness': fitness_score}
    except Exception as e:
        print(f"Error in cluster {cluster_id}: {e}")
        continue

# Display final results
for cluster_id, details in final_routes_퇴근.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Route: {' -> '.join(map(str, details['route']))}")
    print(f"  Fitness Score: {details['fitness']}\n")

Cluster 1 ARS_IDs: ['3165', '3166']
Cluster 1: Start - 3165, End - 3166

[Debug] Inside execute_ga_for_cluster_퇴근
Coords: [[ 37.53549068 127.005729  ]
 [ 37.53284782 127.0083126 ]]
Demand: [20573 11788]
Start Point: 0, End Point: 1
Fitness Value: -1.0
[Cluster Result] Best Route: [0, 1], Fitness: -1.0
Cluster 3 ARS_IDs: ['22013', '22014', '22015', '22019']
Cluster 3: Start - 22013, End - 22019

[Debug] Inside execute_ga_for_cluster_퇴근
Coords: [[ 37.5072265  127.0230319 ]
 [ 37.506367   127.023728  ]
 [ 37.51411252 127.020057  ]
 [ 37.50629005 127.005129  ]]
Demand: [24420 17483 21673 34118]
Start Point: 0, End Point: 3
Generation 1/50 is running...
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800195377
Fitness Value: -1.9645078979588408
Fitness Value: -1.9645078979588408
Fitness Value: -31.833534800195377
Fitness Value: -31.833534800

# 퇴근 유전 알고리즘 결과 시각화

In [46]:
import osmnx as ox
import networkx as nx
import folium

# 지도 중심 설정
map_clusters_퇴근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 색상 리스트 (군집별 구분용)
colors = ['blue', 'green', 'purple', 'orange', 'darkred']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_퇴근.keys())}

# 군집별 경로 시각화
for cluster_id, details in final_routes_퇴근.items():
    route = details['route']
    cluster_data = df_evening[df_evening['ARS_ID'].isin(route)]
    coords = cluster_data[['위도', '경도']].values

    # 도로 네트워크 가져오기
    center_lat_퇴근 = df_evening['위도'].mean()
    center_lon_퇴근 = df_evening['경도'].mean()
    G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=10000, network_type='drive')

    # 도로 경로 생성
    road_route = []
    for i in range(len(coords) - 1):
        # 시작점과 끝점 노드 찾기
        start_node = ox.nearest_nodes(G, X=coords[i][1], Y=coords[i][0])
        end_node = ox.nearest_nodes(G, X=coords[i + 1][1], Y=coords[i + 1][0])

        # 최단 경로 계산
        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            road_route.extend(shortest_path)
        except nx.NetworkXNoPath:
            print(f"No path between {coords[i]} and {coords[i + 1]}")
            continue

    # 중복된 노드 제거
    road_route = list(dict.fromkeys(road_route))

    # 도로 경로 좌표 추출
    road_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in road_route]

    # 도로 경로 추가
    cluster_color = cluster_colors[cluster_id]
    folium.PolyLine(
        locations=road_coords,
        color=cluster_color,
        weight=5,
        opacity=0.8,
        tooltip=f"Cluster {cluster_id} Road Route"
    ).add_to(map_clusters_퇴근)

    # 정류장 마커 추가
    for i, (lat, lon) in enumerate(coords):
        icon_color = 'red' if i == 0 or i == len(coords) - 1 else cluster_color
        folium.Marker(
            location=[lat, lon],
            tooltip=f"Cluster {cluster_id} - Stop {i + 1} (ARS_ID: {route[i]})",
            icon=folium.Icon(color=icon_color, icon="info-sign")
        ).add_to(map_clusters_퇴근)

# 지도 저장
map_clusters_퇴근.save("optimized_cluster_routes_map_퇴근.html")
print("지도 저장 완료: 'optimized_cluster_routes_map_퇴근.html'")
map_clusters_퇴근

IndexError: list index out of range

# 퇴근 유전 알고리즘 시각화 수정 버전

In [47]:
import osmnx as ox
import networkx as nx
import folium
from geopy.distance import geodesic

# 지도 초기화
map_clusters_퇴근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 색상 리스트 (군집별 구분용)
colors = ['blue', 'green', 'purple', 'orange', 'darkred']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_퇴근.keys())}

# 도로 네트워크 가져오기 (모든 정류장이 포함되도록 범위 확장)
center_lat_퇴근 = df_evening['위도'].mean()
center_lon_퇴근 = df_evening['경도'].mean()
G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=10000, network_type='drive')

# 군집별 경로 시각화
for cluster_id, details in final_routes_퇴근.items():
    route = details['route']
    cluster_data = df_evening[df_evening['ARS_ID'].isin(route)]

    # 경로가 없는 경우 스킵
    if cluster_data.empty:
        print(f"Cluster {cluster_id}: No valid stops found.")
        continue

    # 정류장 순서를 클러스터 내 경로 순서에 맞게 정렬
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')

    coords = cluster_data[['위도', '경도']].values

    # 도로 경로 생성
    road_route = []
    for i in range(len(coords) - 1):
        # 시작점과 끝점 노드 찾기
        start_node = ox.distance.nearest_nodes(G, X=coords[i][1], Y=coords[i][0])
        end_node = ox.distance.nearest_nodes(G, X=coords[i + 1][1], Y=coords[i + 1][0])

        # 최단 경로 계산
        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            road_route.extend(shortest_path)
        except nx.NetworkXNoPath:
            print(f"Cluster {cluster_id}: No path between {coords[i]} and {coords[i + 1]}. Using direct line.")
            # 경로가 없으면 직선 연결
            road_route.append((coords[i][0], coords[i][1]))
            road_route.append((coords[i + 1][0], coords[i + 1][1]))

    # 중복된 노드 제거
    road_route = list(dict.fromkeys(road_route))

    # 도로 경로 좌표 추출
    road_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in road_route]

    # 도로 경로 추가
    cluster_color = cluster_colors[cluster_id]
    folium.PolyLine(
        locations=road_coords,
        color=cluster_color,
        weight=5,
        opacity=0.8,
        tooltip=f"Cluster {cluster_id} Road Route"
    ).add_to(map_clusters_퇴근)

    # 정류장 마커 추가
    for _, row in cluster_data.iterrows():
        icon_color = 'red' if row['순서'] == 0 or row['순서'] == len(route) - 1 else cluster_color
        folium.Marker(
            location=[row['위도'], row['경도']],
            tooltip=f"Cluster {cluster_id} - Stop {row['순서'] + 1} (ARS_ID: {row['ARS_ID']})",
            icon=folium.Icon(color=icon_color, icon="info-sign")
        ).add_to(map_clusters_퇴근)

# 지도 저장
map_clusters_퇴근.save("optimized_cluster_routes_map_퇴근.html")
print("지도 저장 완료: 'optimized_cluster_routes_map_퇴근.html'")
map_clusters_퇴근

<ipython-input-47-4779d7d13b15>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: route.index(x))
<ipython-input-47-4779d7d13b15>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: route.index(x))
<ipython-input-47-4779d7d13b15>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

지도 저장 완료: 'optimized_cluster_routes_map_퇴근.html'


# 퇴근 최종 노선

In [49]:
import osmnx as ox
import networkx as nx
import folium
from folium.features import DivIcon

# 지도 초기화
final_map_퇴근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 도로 네트워크 가져오기
center_lat_퇴근 = df_evening['위도'].mean()
center_lon_퇴근 = df_evening['경도'].mean()
G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=10000, network_type='drive')

# 색상 설정
colors = ['blue', 'green', 'purple', 'orange', 'cyan']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_퇴근.keys())}

# 최종 노선 경로 저장
final_route_coords_퇴근 = []  # 도로 경로 포함 전체 경로
all_stop_coords = []  # 정류장 마커 표시용

# 클러스터 순서대로 경로 연결
clusters_order = list(final_routes_퇴근.keys())  # 클러스터 순서
for i, cluster_id in enumerate(clusters_order):
    # 현재 클러스터의 경로와 좌표 가져오기
    cluster_route = final_routes_퇴근[cluster_id]['route']
    cluster_data = df_evening[df_evening['ARS_ID'].isin(cluster_route)].copy()

    # 클러스터 내 정류장 순서 유지
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: cluster_route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')
    cluster_coords = cluster_data[['위도', '경도']].values

    # 클러스터 내 경로를 지도에 추가 (도로 경로로 표시)
    for j in range(len(cluster_coords) - 1):
        start_node = ox.distance.nearest_nodes(G, X=cluster_coords[j][1], Y=cluster_coords[j][0])
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[j + 1][1], Y=cluster_coords[j + 1][0])

        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_퇴근.extend(shortest_path_coords)

            # 클러스터 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color=cluster_colors[cluster_id],
                weight=5,
                opacity=0.8,
                tooltip=f"Cluster {cluster_id} Route"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between stops {j} and {j + 1} in Cluster {cluster_id}.")
            continue

    # 정류장 마커 추가
    all_stop_coords.extend(cluster_coords)

    # 클러스터 간 연결 (현재 클러스터 끝점과 다음 클러스터 시작점)
    if i < len(clusters_order) - 1:
        next_cluster_id = clusters_order[i + 1]
        next_cluster_route = final_routes_퇴근[next_cluster_id]['route']
        next_cluster_data = df_evening[df_evening['ARS_ID'].isin(next_cluster_route)].copy()
        next_cluster_data['순서'] = next_cluster_data['ARS_ID'].apply(lambda x: next_cluster_route.index(x))
        next_cluster_data = next_cluster_data.sort_values(by='순서')
        next_cluster_coords = next_cluster_data[['위도', '경도']].values

        # 현재 클러스터 끝점과 다음 클러스터 시작점 연결
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[-1][1], Y=cluster_coords[-1][0])
        start_node = ox.distance.nearest_nodes(G, X=next_cluster_coords[0][1], Y=next_cluster_coords[0][0])

        try:
            shortest_path = nx.shortest_path(G, end_node, start_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_퇴근.extend(shortest_path_coords)

            # 클러스터 간 연결 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color='black',
                weight=5,
                opacity=0.6,
                tooltip=f"Connection between Cluster {cluster_id} and {next_cluster_id}"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between Cluster {cluster_id} and Cluster {next_cluster_id}.")
            continue

# 정류장 마커 추가 (모든 정류장 표시)
for i, coord in enumerate(all_stop_coords):
    marker_color = 'red' if i == 0 or i == len(all_stop_coords) - 1 else 'blue'
    folium.CircleMarker(
        location=coord,
        radius=6,
        color=marker_color,
        fill=True,
        fill_opacity=0.7,
        tooltip=f"Stop {i + 1}"
    ).add_to(final_map_퇴근)

# 지도 저장
final_map_퇴근.save("final_cluster_route_map_퇴근.html")
print("지도 저장 완료: 'final_cluster_route_map_퇴근.html'")
final_map_퇴근

지도 저장 완료: 'final_cluster_route_map_퇴근.html'


# 퇴근 최종 노선 시각화 포인트 수정

In [50]:
import osmnx as ox
import networkx as nx
import folium
from folium.features import DivIcon

# 지도 초기화
final_map_퇴근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 도로 네트워크 가져오기
center_lat_퇴근 = df_evening['위도'].mean()
center_lon_퇴근 = df_evening['경도'].mean()
G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=10000, network_type='drive')
# 색상 설정
colors = ['blue', 'green', 'purple', 'orange']
cluster_colors = {cluster_id: colors[i % len(colors)] for i, cluster_id in enumerate(final_routes_퇴근.keys())}

# 최종 노선 경로 저장
final_route_coords_퇴근 = []  # 도로 경로 포함 전체 경로
all_stop_coords = []  # 정류장 마커 표시용

# 정류장 순서 저장
stop_order = 1  # 전체 정류장 순서 시작

# 클러스터 순서대로 경로 연결
clusters_order = list(final_routes_퇴근.keys())  # 클러스터 순서
for i, cluster_id in enumerate(clusters_order):
    # 현재 클러스터의 경로와 좌표 가져오기
    cluster_route = final_routes_퇴근[cluster_id]['route']
    cluster_data = df_evening[df_evening['ARS_ID'].isin(cluster_route)].copy()

    # 클러스터 내 정류장 순서 유지
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: cluster_route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')
    cluster_coords = cluster_data[['위도', '경도']].values

    # 클러스터 내 경로를 지도에 추가 (도로 경로로 표시)
    for j in range(len(cluster_coords) - 1):
        start_node = ox.distance.nearest_nodes(G, X=cluster_coords[j][1], Y=cluster_coords[j][0])
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[j + 1][1], Y=cluster_coords[j + 1][0])

        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color=cluster_colors[cluster_id],
                weight=5,
                opacity=0.8,
                tooltip=f"Cluster {cluster_id} Route"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between stops {j} and {j + 1} in Cluster {cluster_id}.")
            continue

    # 정류장 마커 추가 (순서 포함)
    for coord in cluster_coords:
        # 마커 추가 (포인트 크기 증가)
        marker_color = 'red' if stop_order == 1 or stop_order == len(all_stop_coords) + len(cluster_coords) else 'blue'
        folium.CircleMarker(
            location=coord,
            radius=12,  # 마커 크기 증가
            color=marker_color,
            fill=True,
            fill_opacity=0.7
        ).add_to(final_map_퇴근)

        # 정류장 순서 텍스트 추가
        folium.map.Marker(
            location=coord,
            icon=DivIcon(
                icon_size=(200, 36),
                icon_anchor=(0, 0),
                html=(
                    f'<div style="font-size: 16px; color: white; font-weight: bold; '
                    f'text-shadow: -1px -1px 0 black, 1px -1px 0 black, -1px 1px 0 black, 1px 1px 0 black;">'
                    f'{stop_order}</div>'
                )
            )
        ).add_to(final_map_퇴근)

        stop_order += 1

    # 다음 클러스터와 연결
    if i < len(clusters_order) - 1:
        next_cluster_id = clusters_order[i + 1]
        next_cluster_route = final_routes_퇴근[next_cluster_id]['route']
        next_cluster_data = df_evening[df_evening['ARS_ID'].isin(next_cluster_route)].copy()
        next_cluster_data['순서'] = next_cluster_data['ARS_ID'].apply(lambda x: next_cluster_route.index(x))
        next_cluster_data = next_cluster_data.sort_values(by='순서')
        next_cluster_coords = next_cluster_data[['위도', '경도']].values

        # 현재 클러스터 끝점과 다음 클러스터 시작점 연결
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[-1][1], Y=cluster_coords[-1][0])
        start_node = ox.distance.nearest_nodes(G, X=next_cluster_coords[0][1], Y=next_cluster_coords[0][0])

        try:
            shortest_path = nx.shortest_path(G, end_node, start_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_출근.extend(shortest_path_coords)

            # 클러스터 간 연결 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color='black',
                weight=5,
                opacity=0.6,
                tooltip=f"Connection between Cluster {cluster_id} and {next_cluster_id}"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between Cluster {cluster_id} and Cluster {next_cluster_id}.")
            continue

# 지도 저장
final_map_퇴근.save("final_cluster_route_map_퇴근.html")
print("지도 저장 완료: 'final_cluster_route_map_퇴근.html'")
final_map_퇴근


지도 저장 완료: 'final_cluster_route_map_퇴근.html'


# 퇴근 최종 노선 시각화 수정

In [51]:
import osmnx as ox
import networkx as nx
import folium
from folium.features import DivIcon

# 지도 초기화
final_map_퇴근 = folium.Map(location=[37.5, 127.0], zoom_start=13)

# 도로 네트워크 가져오기
center_lat_퇴근 = df_evening['위도'].mean()
center_lon_퇴근 = df_evening['경도'].mean()
G = ox.graph_from_point((center_lat_퇴근, center_lon_퇴근), dist=10000, network_type='drive')

# 정류장 순서 저장
stop_order = 1  # 전체 정류장 순서 시작
total_first_stop = True  # 전체 첫 정류장 플래그
total_last_stop_coords = None  # 전체 마지막 정류장 좌표 저장

# 클러스터 순서대로 경로 연결
clusters_order = list(final_routes_퇴근.keys())  # 클러스터 순서
for i, cluster_id in enumerate(clusters_order):
    # 현재 클러스터의 경로와 좌표 가져오기
    cluster_route = final_routes_퇴근[cluster_id]['route']
    cluster_data = df_evening[df_evening['ARS_ID'].isin(cluster_route)].copy()

    # 클러스터 내 정류장 순서 유지
    cluster_data['순서'] = cluster_data['ARS_ID'].apply(lambda x: cluster_route.index(x))
    cluster_data = cluster_data.sort_values(by='순서')
    cluster_coords = cluster_data[['위도', '경도']].values

    # 클러스터 내 경로를 지도에 추가 (도로 경로로 표시)
    for j in range(len(cluster_coords) - 1):
        start_node = ox.distance.nearest_nodes(G, X=cluster_coords[j][1], Y=cluster_coords[j][0])
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[j + 1][1], Y=cluster_coords[j + 1][0])

        try:
            shortest_path = nx.shortest_path(G, start_node, end_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_퇴근.extend(shortest_path_coords)

            # 클러스터 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color=cluster_colors[cluster_id],
                weight=5,
                opacity=0.8,
                tooltip=f"Cluster {cluster_id} Route"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between stops {j} and {j + 1} in Cluster {cluster_id}.")
            continue

    # 정류장 마커 추가 (전체 첫 정류장과 마지막 정류장 처리)
    for idx, coord in enumerate(cluster_coords):
        if total_first_stop:  # 전체 첫 정류장
            marker_color = 'red'
            total_first_stop = False
        elif i == len(clusters_order) - 1 and idx == len(cluster_coords) - 1:  # 전체 마지막 정류장
            marker_color = 'red'
        else:  # 중간 정류장
            marker_color = 'blue'

        # 마커 추가 (포인트 크기 증가)
        folium.CircleMarker(
            location=coord,
            radius=12,  # 마커 크기 증가
            color=marker_color,
            fill=True,
            fill_opacity=0.7
        ).add_to(final_map_퇴근)

        # 정류장 순서 텍스트 추가
        folium.map.Marker(
            location=coord,
            icon=DivIcon(
                icon_size=(200, 36),
                icon_anchor=(12, 12),  # 텍스트 중심 위치
                html=(f'<div style="font-size: 16px; color: white; font-weight: bold; '
                      f'text-shadow: -1px -1px 0 black, 1px -1px 0 black, -1px 1px 0 black, 1px 1px 0 black;">'
                      f'{stop_order}</div>')
            )
        ).add_to(final_map_퇴근)

        stop_order += 1

    # 다음 클러스터와 연결
    if i < len(clusters_order) - 1:
        next_cluster_id = clusters_order[i + 1]
        next_cluster_route = final_routes_퇴근[next_cluster_id]['route']
        next_cluster_data = df_evening[df_evening['ARS_ID'].isin(next_cluster_route)].copy()
        next_cluster_data['순서'] = next_cluster_data['ARS_ID'].apply(lambda x: next_cluster_route.index(x))
        next_cluster_data = next_cluster_data.sort_values(by='순서')
        next_cluster_coords = next_cluster_data[['위도', '경도']].values

        # 현재 클러스터 끝점과 다음 클러스터 시작점 연결
        end_node = ox.distance.nearest_nodes(G, X=cluster_coords[-1][1], Y=cluster_coords[-1][0])
        start_node = ox.distance.nearest_nodes(G, X=next_cluster_coords[0][1], Y=next_cluster_coords[0][0])

        try:
            shortest_path = nx.shortest_path(G, end_node, start_node, weight="length")
            shortest_path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
            final_route_coords_퇴근.extend(shortest_path_coords)

            # 클러스터 간 연결 경로 시각화
            folium.PolyLine(
                locations=shortest_path_coords,
                color='black',
                weight=5,
                opacity=0.6,
                tooltip=f"Connection between Cluster {cluster_id} and {next_cluster_id}"
            ).add_to(final_map_퇴근)
        except nx.NetworkXNoPath:
            print(f"No path between Cluster {cluster_id} and Cluster {next_cluster_id}.")
            continue

# 지도 저장
final_map_퇴근.save("final_cluster_route_map_퇴근.html")
print("지도 저장 완료: 'final_route_map_퇴근.html'")
final_map_퇴근

지도 저장 완료: 'final_route_map_퇴근.html'


## 기초 코드

In [ ]:
# Apply GA to each cluster with user-defined start and end points
final_routes = {}

start_ends = {
    2: {'start': '22293', 'end': '22004'},
    0: {'start': '23281', 'end': '23282'},
    4: {'start': '23283', 'end': '23286'},
    1: {'start': '22014', 'end': '22015'},
    3: {'start': '3166', 'end': '3165'}
}

# Ensure ARS_ID and start_ends values are strings
df_evening['ARS_ID'] = df_evening['ARS_ID'].astype(str)
start_ends = {k: {'start': str(v['start']), 'end': str(v['end'])} for k, v in start_ends.items()}

for cluster_id in route_퇴근:
    # Filter current cluster data
    current_cluster = df_evening[df_evening['클러스터'] == cluster_id].reset_index(drop=True)

    # Debug: Check cluster ARS_IDs
    print(f"Cluster {cluster_id} ARS_IDs: {current_cluster['ARS_ID'].tolist()}")

    # Find start and end points based on user-defined `start_ends`
    start_matches = current_cluster[current_cluster['ARS_ID'] == start_ends[cluster_id]['start']]
    end_matches = current_cluster[current_cluster['ARS_ID'] == start_ends[cluster_id]['end']]

    # Ensure start and end points are valid
    if start_matches.empty:
        print(f"Warning: Start point {start_ends[cluster_id]['start']} not found in cluster {cluster_id}.")
        start_point = 0  # Default to first point
    else:
        start_point = start_matches.index[0]

    if end_matches.empty:
        print(f"Warning: End point {start_ends[cluster_id]['end']} not found in cluster {cluster_id}.")
        end_point = len(current_cluster) - 1  # Default to last point
    else:
        end_point = end_matches.index[0]

    # Debug: Check assigned start and end points
    print(f"Cluster {cluster_id}: Start - {current_cluster.iloc[start_point]['ARS_ID']}, "
          f"End - {current_cluster.iloc[end_point]['ARS_ID']}")

    # Filter middle stations
    middle_stations = [i for i in range(len(current_cluster)) if i != start_point and i != end_point]
    if not middle_stations:
        print(f"Warning: No valid middle stations for cluster {cluster_id}. Using start and end points only.")
        final_routes[cluster_id] = {'route': [start_ends[cluster_id]['start'], start_ends[cluster_id]['end']],
                                    'fitness': 0}
        continue

    # Execute genetic algorithm for the cluster
    try:
        route_ids, fitness_score = execute_ga_for_cluster(current_cluster, distance_threshold, start_point, end_point)
        final_routes[cluster_id] = {'route': route_ids, 'fitness': fitness_score}
    except IndexError as e:
        print(f"Error in cluster {cluster_id}: {e}")
        continue

# Display final results
for cluster_id, details in final_routes.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Route: {' -> '.join(map(str, details['route']))}")
    print(f"  Fitness Score: {details['fitness']}\n")


Cluster 2 ARS_IDs: ['22004', '22293']
Cluster 2: Start - 22293, End - 22004
Cluster 0 ARS_IDs: ['23275', '23276', '23277', '23278', '23280', '23281', '23282']
Cluster 0: Start - 23281, End - 23282
Cluster 4 ARS_IDs: ['22008', '22010', '22012', '23283', '23284', '23285', '23286', '23287', '22007', '22009', '22011']
Cluster 4: Start - 23283, End - 23286
Cluster 1 ARS_IDs: ['22013', '22014', '22015', '22019']
Cluster 1: Start - 22014, End - 22015
Cluster 3 ARS_IDs: ['3165', '3166']
Cluster 3: Start - 3166, End - 3165
Cluster 2:
  Route: 22293 -> 22004
  Fitness Score: 0

Cluster 0:
  Route: 23281 -> 23278 -> 23275 -> 23276 -> 23277 -> 23280 -> 23282
  Fitness Score: -101602.4433525131

Cluster 4:
  Route: 23283 -> 23284 -> 22007 -> 22008 -> 22010 -> 22009 -> 23287 -> 23285 -> 22011 -> 22012 -> 23286
  Fitness Score: -446209.08478749904

Cluster 1:
  Route: 22014 -> 22013 -> 22019 -> 22015
  Fitness Score: -199074.12459562166

Cluster 3:
  Route: 3166 -> 3165
  Fitness Score: 0

